# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression
Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

>*This is a classification problem, since we are trying to classify students into two categories: passed and not passed.*

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [539]:
# Import libraries
import numpy as np
import pandas as pd
from ipy_table import make_table, set_row_style

In [540]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [541]:
from __future__ import division

n_students, n_features = student_data.shape
n_features -= 1 # Subtract 1 (label col).
n_passed = len(student_data[student_data.passed == 'yes'])
n_failed = len(student_data[student_data.passed == 'no'])
grad_rate = (n_passed*100)/(n_students)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [542]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [543]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [544]:
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=42)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model. (see table below)

In [545]:
# Dictionary to store statistics related to models so we can make a table later.
from collections import defaultdict
model_stats = defaultdict(lambda: defaultdict(dict))

In [546]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    model_type = clf.__class__.__name__
    size = len(X_train)
    print "Training {}...".format(model_type)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    total_time = end - start
    model_stats[model_type][size]['Training time'] = '{:.3f}'.format(total_time)
    print "Done!\nTraining time (secs): {:.3f}".format(total_time)

In [547]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    model_type = clf.__class__.__name__
    size = len(features)
    print "Predicting labels using {}...".format(model_type)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    total_time = end - start
    # Collect stats for this model
    model_stats[model_type][size]['Prediction time'] = '{:.3f}'.format(total_time)
    print "Done!\nPrediction time (secs): {:.3f}".format(total_time)
    return f1_score(target.values, y_pred, pos_label='yes')

In [548]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    model_type = clf.__class__.__name__
    size = len(X_train)
    print "------------------------------------------"
    print "Training set size: {}".format(size)
    train_classifier(clf, X_train, y_train)
    
    f1_train = predict_labels(clf, X_train, y_train)
    f1_test = predict_labels(clf, X_test, y_test)
    # Collect stats for this model
    model_stats[model_type][size]['f1 score for training'] = f1_train
    model_stats[model_type][size]['f1 score for test'] = f1_test
    
    print "F1 score for training set: {}".format(f1_train)
    print "F1 score for test set: {}".format(f1_test)

In [549]:
# Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
X_train_200, X_test_dummy, y_train_200, y_test_dummy = train_test_split(X_train, y_train, train_size=200, random_state=42)
X_train_100, X_test_dummy, y_train_100, y_test_dummy = train_test_split(X_train, y_train, train_size=100, random_state=42)

In [550]:
# Train and predict using SVC
# Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC(kernel='rbf')
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train_200, y_train_200, X_test, y_test)
train_predict(clf, X_train_100, y_train_100, X_test, y_test)

------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.015
Predicting labels using SVC...
Done!
Prediction time (secs): 0.009
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.876068376068
F1 score for test set: 0.783783783784
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.008
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.890322580645
F1 score for test set: 0.797297297297
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.904458598726
F1 score for test set: 0.76821192053


In [551]:
# Train and predict using two other models
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train_200, y_train_200, X_test, y_test)
train_predict(clf, X_train_100, y_train_100, X_test, y_test)

------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.003
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.80378250591
F1 score for test set: 0.763358778626
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.828571428571
F1 score for test set: 0.753846153846
------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0

In [552]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train_200, y_train_200, X_test, y_test)
train_predict(clf, X_train_100, y_train_100, X_test, y_test)

------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.009
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.880898876404
F1 score for test set: 0.780141843972
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.879194630872
F1 score for test set: 0.779411764706
------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.006
Predicting labels usi

In [553]:
# Construct tables with model statistics.
models_to_show = ['SVC', 'KNeighborsClassifier', 'GaussianNB']
for m in models_to_show:
    print m
    print pd.DataFrame(model_stats.get(m)).dropna(axis='columns')
    print '\n'

SVC
                            100       200       300
Prediction time           0.002     0.003     0.009
Training time             0.002     0.008     0.015
f1 score for test      0.768212  0.797297  0.783784
f1 score for training  0.904459  0.890323  0.876068


KNeighborsClassifier
                            100       200       300
Prediction time           0.006     0.004     0.009
Training time             0.001     0.001     0.002
f1 score for test      0.772414  0.779412  0.780142
f1 score for training      0.88  0.879195  0.880899


GaussianNB
                            100       200       300
Prediction time           0.000     0.000     0.001
Training time             0.001     0.001     0.003
f1 score for test      0.404762  0.753846  0.763359
f1 score for training  0.325581  0.828571  0.803783




## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [554]:
from sklearn import grid_search

# Run Grid Search with SVM to find the best C, gamma and kernel values.
param_grid = [
    {'C': [1, 2, 4, 8, 64, 128],
     'gamma': [1e-4, 1e-3, 1e-2, 0.1, 1],
     'kernel': ['rbf']},
    {'C': [1, 2, 5, 10],
     'kernel': ['linear']}
]

clf = grid_search.GridSearchCV(svm.SVC(), param_grid, cv=10, n_jobs=3)
clf.fit(X_train, y_train)

print "Parameters for the best estimator:\n", clf.best_estimator_
print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Parameters for the best estimator:
SVC(C=8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.842975206612
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.783783783784
